In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
DATA_DIR = '/content/drive/Shareddrives/Intelligent System - Assignment /Data/'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

In [ ]:
gensim.__version__

'3.6.0'

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
data = pd.read_csv(DATA_DIR+'guardians-news/combined_filtered.csv')
data.head()

,Unnamed: 0,apiUrl,fields.bodyText,id,isHosted,pillarId,pillarName,sectionId,sectionName,type,webPublicationDate,webTitle,webUrl,filtered_bodyText
0,0,https://content.guardianapis.com/technology/20...,A day after Barack Obama announced tough new s...,technology/2016/dec/31/russia-hacking-malware-...,False,pillar/news,News,technology,Technology,article,2016-12-31T16:34:02Z,US-Russia tensions rise as malware found at Ve...,https://www.theguardian.com/technology/2016/de...,"['day', 'barack', 'obama', 'announced', 'tough..."
1,1,https://content.guardianapis.com/technology/20...,A journalist was temporarily banned from Faceb...,technology/2016/dec/30/facebook-temporary-ban-...,False,pillar/news,News,technology,Technology,article,2016-12-30T21:22:06Z,Facebook temporarily bans author after he call...,https://www.theguardian.com/technology/2016/de...,"['journalist', 'temporarily', 'banned', 'faceb..."
2,2,https://content.guardianapis.com/technology/20...,Technology? Bah humbug: “I think we ought to g...,technology/2016/dec/30/donald-trump-technology...,False,pillar/news,News,technology,Technology,article,2016-12-30T08:30:28Z,Donald Trump is technology's befuddled (but da...,https://www.theguardian.com/technology/2016/de...,"['technology', 'bah', 'humbug', 'think', 'ough..."
3,3,https://content.guardianapis.com/technology/20...,While there were some good things in technolog...,technology/2016/dec/30/detonating-devices-and-...,False,pillar/news,News,technology,Technology,article,2016-12-30T07:00:26Z,Exploding phones and Snapchat clones: the bigg...,https://www.theguardian.com/technology/2016/de...,"['good', 'things', 'technology', 'released', '..."
4,4,https://content.guardianapis.com/technology/20...,The US Department of Homeland Security (DHS) a...,technology/2016/dec/29/fbi-dhs-russian-hacking...,False,pillar/news,News,technology,Technology,article,2016-12-29T22:19:53Z,FBI and Homeland Security detail Russian hacki...,https://www.theguardian.com/technology/2016/de...,"['us', 'department', 'homeland', 'security', '..."


In [ ]:
data.shape

(52900, 14)

In [ ]:
def preprocessing(inp):
  text = str(inp)
  tokens = [re.sub(r'[^a-z|A-Z]', ' ', word.strip().lower()) for word in text.split()]

  return " ".join(tokens)

In [ ]:
data['fields.bodyText'][0]

'A day after Barack Obama announced tough new sanctions over what intelligence agencies believe to be Russian attempts to influence the presidential election in favour of Donald Trump, US officials said computer code linked to Russian-sponsored hackers had been detected in a computer at a Vermont electric utility. The municipally run Burlington electric department confirmed on Friday that it had found, in a laptop not connected to grid systems, malware code used in Grizzly Steppe, the name the Department of Homeland Security (DHS) and FBI have applied to a Russian campaign linked to cyber-attacks on the Democratic National Committee and other political organizations. The Washington Post first reported the discovery. On Thursday, the day on which Obama announced the new sanctions, the DHS and the FBI published a report detailing what they called Russia’s “ongoing campaign of cyber-enabled operations directed at the US government and its citizens”. After the discovery in Vermont, however

In [ ]:
preprocessing(preprocessing(data.loc[0, 'fields.bodyText']))

'a day after barack obama announced tough new sanctions over what intelligence agencies believe to be russian attempts to influence the presidential election in favour of donald trump us officials said computer code linked to russian sponsored hackers had been detected in a computer at a vermont electric utility the municipally run burlington electric department confirmed on friday that it had found in a laptop not connected to grid systems malware code used in grizzly steppe the name the department of homeland security dhs and fbi have applied to a russian campaign linked to cyber attacks on the democratic national committee and other political organizations the washington post first reported the discovery on thursday the day on which obama announced the new sanctions the dhs and the fbi published a report detailing what they called russia s ongoing campaign of cyber enabled operations directed at the us government and its citizens after the discovery in vermont however officials said

In [ ]:
data['text'] = data['webTitle'] + ' ' + data['fields.bodyText']
data['text'] = data['text'].apply(preprocessing).apply(preprocessing)
data['text']

0        us russia tensions rise as malware found at ve...
1        facebook temporarily bans author after he call...
2        donald trump is technology s befuddled but dan...
3        exploding phones and snapchat clones the bigge...
4        fbi and homeland security detail russian hacki...
                               ...                        
52895                                                  nan
52896    top british art galleries investigate sexual m...
52897    stetsons beards and good honest sweat why pop ...
52898    what to see this week in the uk five of the be...
52899    we had death threats the defiant return of wil...
Name: text, Length: 52900, dtype: object

In [ ]:
data[['text', 'sectionName']].to_csv(DATA_DIR+'guardians-news/processed.csv', index=False)

In [ ]:
# text = data['text'].values.tolist()
text = pd.read_csv(DATA_DIR+'guardians-news/processed.csv')
text.head()

,text,sectionName
0,us russia tensions rise as malware found at ve...,Technology
1,facebook temporarily bans author after he call...,Technology
2,donald trump is technology s befuddled but dan...,Technology
3,exploding phones and snapchat clones the bigge...,Technology
4,fbi and homeland security detail russian hacki...,Technology


In [ ]:
X = data['text'].values

In [ ]:
LE = LabelEncoder()
y = pd.get_dummies(text['sectionName'])
display(y)
y = y.values

,Business,Culture,Politics,Sport,Technology
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
52895,0,1,0,0,0
52896,0,1,0,0,0
52897,0,1,0,0,0
52898,0,1,0,0,0


## 1. Doc2Vec + CNN

In [ ]:
max_epochs = 5
vec_size = 50

In [ ]:
tagged_data = [TaggedDocument(words=str(_d), tags=[str(i)]) for i, _d in enumerate(text.values.tolist())]

In [ ]:
# model = Doc2Vec(vector_size=vec_size,
#                 min_count=5,
#                 dm=1,
#                 window=3,
#                 epochs=10)
  
# model.build_vocab(tagged_data)

# for epoch in range(max_epochs):
#     print('iteration {0}'.format(epoch))
#     model.train(tagged_data,
#                 total_examples=model.corpus_count,
#                 epochs=model.epochs)
#     # decrease the learning rate
#     model.alpha -= 0.002
#     # fix the learning rate, no decay
#     model.min_alpha = model.alpha

# model.save("d2v.model")
# print("Model Saved")

In [ ]:
t = text.iloc[:, 0]

In [ ]:
model = Doc2Vec.load("/content/drive/Shareddrives/Intelligent System - Assignment /Models/d2v.model")
v1 = model.infer_vector(t[0])
v1

array([-2.6276822 , -5.642323  ,  2.986788  ,  0.9726205 , -1.6073532 ,
        1.9135802 ,  2.6972883 ,  1.7701771 ,  5.621737  ,  0.7881936 ,
       -2.7089293 , -0.11184725,  3.9315314 , -0.5227014 , -2.027259  ,
        2.3700554 , -4.0916357 ,  5.773399  ,  6.7118716 ,  3.9832294 ,
        2.1616256 ,  5.695961  , -0.75734735,  2.0524323 , -1.256631  ,
        6.9899864 ,  1.3326321 , 11.657165  ,  0.9683079 ,  6.182285  ,
        0.6348038 , -3.725156  ,  1.4782498 , -2.6773195 ,  4.6924586 ,
        0.63313407, -5.157064  ,  2.0772688 ,  1.6384289 ,  0.22722006,
       -1.250312  ,  1.6568228 , -0.889915  , -3.2801628 ,  1.8158349 ,
        4.0833697 , -0.4578227 , -3.5708444 ,  4.9870257 , -1.4840081 ],
      dtype=float32)

In [ ]:
input_matrix = np.zeros((text.shape[0], vec_size))

In [ ]:
for i, d in enumerate(t):
  input_matrix[i] = model.infer_vector(str(d).split())

In [ ]:
X = input_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42320, 50), (10580, 50), (42320, 5), (10580, 5))

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=(50, 1)),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 48, 16)            64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 16)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 32)            1568      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 4, 64)            0

In [ ]:
model.fit(X_train[:, :, np.newaxis], y_train, epochs=100, batch_size=256)

Epoch 1/100
166/166 [==============================] - 15s 10ms/step - loss: 0.5444 - accuracy: 0.8011
Epoch 2/100
166/166 [==============================] - 2s 9ms/step - loss: 0.2779 - accuracy: 0.9042
Epoch 3/100
166/166 [==============================] - 2s 9ms/step - loss: 0.2387 - accuracy: 0.9161
Epoch 4/100
166/166 [==============================] - 2s 9ms/step - loss: 0.2182 - accuracy: 0.9230
Epoch 5/100
166/166 [==============================] - 2s 9ms/step - loss: 0.2025 - accuracy: 0.9280
Epoch 6/100
166/166 [==============================] - 2s 9ms/step - loss: 0.1923 - accuracy: 0.9315
Epoch 7/100
166/166 [==============================] - 2s 9ms/step - loss: 0.1819 - accuracy: 0.9354
Epoch 8/100
166/166 [==============================] - 2s 9ms/step - loss: 0.1743 - accuracy: 0.9370
Epoch 9/100
166/166 [==============================] - 2s 9ms/step - loss: 0.1656 - accuracy: 0.9404
Epoch 10/100
166/166 [==============================] - 2s 9ms/step - loss: 0.1603 - accu

In [ ]:
y_pred = model.predict(X_test[:, :, np.newaxis])
y_pred

array([[1.1366751e-18, 2.9912561e-34, 9.2850098e-30, 1.0000000e+00,
        2.6331306e-31],
       [9.9999976e-01, 5.2738222e-12, 1.7972401e-07, 8.6918706e-10,
        6.8524559e-13],
       [1.3414286e-12, 3.2808657e-21, 5.6112692e-23, 1.0000000e+00,
        3.5001632e-26],
       ...,
       [2.8106463e-06, 4.4945061e-08, 9.9998736e-01, 1.5966403e-08,
        9.8131140e-06],
       [0.0000000e+00, 4.6635130e-36, 2.7127593e-31, 1.0000000e+00,
        0.0000000e+00],
       [1.4380906e-15, 2.6015718e-05, 4.0855495e-15, 3.2969370e-15,
        9.9997401e-01]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      2374
           1       0.81      0.77      0.79       525
           2       0.94      0.91      0.93      2549
           3       0.98      0.99      0.98      4091
           4       0.87      0.88      0.87      1041

    accuracy                           0.93     10580
   macro avg       0.90      0.89      0.90     10580
weighted avg       0.93      0.93      0.93     10580



In [ ]:
model.evaluate(X_test[:, :, np.newaxis], y_test)

331/331 [==============================] - 2s 5ms/step - loss: 0.4558 - accuracy: 0.9310


[0.45578497648239136, 0.9310019016265869]

In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 1 0 0 0]
[0.03359407 0.28800184 0.17531124 0.48357198 0.01952084]
Wrong sample 1
[0 0 0 0 1]
[1.0000000e+00 7.0492184e-12 1.7319680e-13 6.9305256e-12 2.9660449e-10]
Wrong sample 2
[0 1 0 0 0]
[2.7075328e-20 2.2605926e-02 2.9822887e-12 5.4466681e-10 9.7739410e-01]
Wrong sample 3
[0 0 1 0 0]
[9.6818292e-01 4.8401105e-08 3.0565511e-02 3.5179207e-07 1.2512638e-03]
Wrong sample 4
[0 0 0 0 1]
[1.0000000e+00 1.3875386e-12 4.3644564e-11 5.0007691e-09 2.8160739e-14]
Wrong sample 5
[0 1 0 0 0]
[0.04090134 0.24902835 0.18475328 0.50821537 0.01710167]
Wrong sample 6
[1 0 0 0 0]
[2.5128207e-01 4.6620676e-06 7.4648607e-01 2.1885387e-03 3.8611313e-05]
Wrong sample 7
[0 0 0 0 1]
[5.5474877e-01 5.3839381e-06 3.0210271e-07 2.5782504e-04 4.4498780e-01]
Wrong sample 8
[1 0 0 0 0]
[3.4355679e-01 1.6851000e-10 6.5644324e-01 1.6238617e-10 2.4913653e-11]
Wrong sample 9
[0 0 1 0 0]
[6.1929118e-06 9.2155629e-01 7.8433275e-02 6.2493962e-07 3.6295180e-06]
Wrong sample 10
[1 0 0 0 0]
[2.1873246e-

In [ ]:
model.save_weights('./model/doc2vec_checkpoint')

## 2. GloVe + CNN

In [ ]:
X = text.iloc[:, 0].astype(str)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

43199    the golden vision caribbean cricket and europe...
35093    horse racing tips tuesday may hexham another b...
29913    craig sager beloved nba broadcaster dies aged ...
27654    barclays half year profits fall following sett...
31145    dartmouth finishes second for the queen in can...
                               ...                        
52724    on my radar anya taylor joy s cultural highlig...
46713    talking horses john gosden s record points to ...
24942    councils and charities feeling the squeeze | l...
24108    years on from the credit crunch share your mem...
12219    caroline lucas urges jeremy corbyn to discuss ...
Name: text, Length: 42320, dtype: object

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

In [ ]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/Shareddrives/Intelligent System - Assignment /Glove/glove.6B.100d.txt')

In [ ]:
maxLen = 500
embed_vector_len = 100
vocab_len = len(words_to_index) + 1

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = tf.keras.preprocessing.sequence.pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
def glove_cnn(filters = 10, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input = tf.keras.Input(shape=(max_length,))
    embedding = tf.keras.layers.Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input)
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(reshape)
    pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters*2, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(pool1)
    pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters*4, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(pool2)
    pool3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv3)
    flat1 = tf.keras.layers.Flatten()(pool3)
    drop1 = tf.keras.layers.Dropout(0.3)(flat1)
    dense1 = tf.keras.layers.Dense(40, activation='relu')(drop1)
    drop2 = tf.keras.layers.Dropout(0.3)(dense1)
    dense2 = tf.keras.layers.Dense(40, activation='relu')(drop2)
    drop3 = tf.keras.layers.Dropout(0.3)(dense2)
    dense3 = tf.keras.layers.Dense(40, activation='relu')(drop3)
    
    # Interpretation
    outputs = tf.keras.layers.Dense(5, activation='softmax')(dense3)
    model = tf.keras.Model(inputs=input, outputs=outputs)
    
    # Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = glove_cnn(input_dim=vocab_len, output_dim=embed_vector_len, emb_matrix=emb_matrix, max_length=maxLen)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 500)]             0         
                                                                 
 embedding_6 (Embedding)     (None, 500, 100)          16269300  
                                                                 
 conv1d_6 (Conv1D)           (None, 498, 10)           3010      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 249, 10)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 247, 20)           620       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 123, 20)          0         
 1D)                                                       

In [ ]:
model.fit(X_train_indices, y_train, batch_size=256, epochs=100)

Epoch 1/100
166/166 [==============================] - 6s 24ms/step - loss: 0.8878 - accuracy: 0.6300
Epoch 2/100
166/166 [==============================] - 4s 22ms/step - loss: 0.4243 - accuracy: 0.8521
Epoch 3/100
166/166 [==============================] - 4s 22ms/step - loss: 0.3359 - accuracy: 0.8901
Epoch 4/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2868 - accuracy: 0.9073
Epoch 5/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2601 - accuracy: 0.9176
Epoch 6/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2452 - accuracy: 0.9227
Epoch 7/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2311 - accuracy: 0.9270
Epoch 8/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2222 - accuracy: 0.9289
Epoch 9/100
166/166 [==============================] - 4s 22ms/step - loss: 0.2073 - accuracy: 0.9339
Epoch 10/100
166/166 [==============================] - 4s 22ms/step - loss: 0.199

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = tf.keras.preprocessing.sequence.pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
y_pred = model.predict(X_test_indices)
y_pred

array([[8.7624741e-01, 1.1473132e-03, 1.1827835e-01, 2.8860828e-04,
        4.0382980e-03],
       [1.2049927e-05, 1.2308620e-09, 9.9998796e-01, 4.4988957e-10,
        1.7737693e-10],
       [1.1657125e-21, 1.5326861e-21, 2.6574936e-21, 1.0000000e+00,
        2.1531446e-23],
       ...,
       [2.5555410e-28, 6.3711465e-31, 2.1747168e-28, 1.0000000e+00,
        1.7241997e-35],
       [9.9999809e-01, 4.2215762e-11, 1.7531289e-06, 8.1097951e-08,
        6.7190392e-08],
       [4.3526123e-04, 8.9923475e-08, 9.9956065e-01, 2.2372082e-07,
        3.6479612e-06]], dtype=float32)

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.88      0.90      2426
           1       0.79      0.88      0.83       504
           2       0.92      0.93      0.93      2550
           3       0.98      0.99      0.98      4105
           4       0.85      0.89      0.87       995

    accuracy                           0.93     10580
   macro avg       0.89      0.91      0.90     10580
weighted avg       0.93      0.93      0.93     10580



In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[1 0 0 0 0]
[5.7479076e-04 9.9155617e-01 5.3019507e-04 3.2929529e-03 4.0457919e-03]
Wrong sample 1
[0 0 0 0 1]
[1.12989520e-08 1.15911485e-08 1.20184929e-09 9.99913692e-01
 8.62629822e-05]
Wrong sample 2
[1 0 0 0 0]
[4.5321766e-01 2.2332713e-10 5.4678237e-01 1.0603456e-09 4.2808614e-09]
Wrong sample 3
[0 1 0 0 0]
[1.0094322e-04 1.9165607e-03 3.4528866e-02 9.6336693e-01 8.6761458e-05]
Wrong sample 4
[0 0 1 0 0]
[7.8768587e-01 3.8531361e-06 2.1228480e-01 4.9410742e-06 2.0569069e-05]
Wrong sample 5
[1 0 0 0 0]
[2.4308470e-01 2.0472024e-05 7.5682777e-01 5.4295765e-06 6.1656647e-05]
Wrong sample 6
[1 0 0 0 0]
[5.3472591e-05 1.6510928e-08 9.9994612e-01 2.4867347e-07 1.2057446e-07]
Wrong sample 7
[1 0 0 0 0]
[3.6645631e-09 4.8050500e-12 1.8965197e-11 5.3981880e-12 1.0000000e+00]
Wrong sample 8
[0 0 1 0 0]
[2.2435426e-03 4.4800835e-03 5.0063372e-02 4.7764748e-05 9.4316524e-01]
Wrong sample 9
[1 0 0 0 0]
[5.3280815e-03 5.5026811e-07 9.9449313e-01 2.6320608e-08 1.7825508e-04]
Wron

In [ ]:
model.evaluate(X_test_indices, y_test)

331/331 [==============================] - 2s 5ms/step - loss: 0.3214 - accuracy: 0.9328


[0.3213595747947693, 0.9327977299690247]